1. 将服务器端每一条新闻转化成txt格式存储

In [18]:
import os
import pandas as pd
import numpy as np

dirs = "./preprocess"
if not os.path.exists(dirs):
    os.mkdir(dirs)

df = pd.read_csv("./data/all_news.csv")
for item in df.iloc():
    with open("{}/{}.txt".format(dirs, item.id), "w") as f:
        f.write(item.title)
        f.write("\n")
        f.write(item.body)

保存每条新闻id和title的对应关系